# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [22]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

# from sklearn.model_selection import train_test_split
# from sklearn import metrics
# from sklearn.ensemble import RandomForestClassifier

# from scipy.stats import ks_2samp
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

%matplotlib inline

In [23]:
renda_prev = pd.read_csv('previsao_de_renda.csv')

In [24]:
renda_prev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.

2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.

3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [25]:
renda_prev.head()

,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [26]:
renda_prev.columns

Index(['Unnamed: 0', 'data_ref', 'id_cliente', 'sexo', 'posse_de_veiculo',
       'posse_de_imovel', 'qtd_filhos', 'tipo_renda', 'educacao',
       'estado_civil', 'tipo_residencia', 'idade', 'tempo_emprego',
       'qt_pessoas_residencia', 'renda'],
      dtype='object')

In [42]:
y, x = patsy.dmatrices('renda ~ C(data_ref) + id_cliente + C(sexo) + posse_de_veiculo + posse_de_imovel +'
                       ' qtd_filhos + C(tipo_renda) + C(educacao) + C(estado_civil, Treatment()) + '
                       'C(tipo_residencia) + idade + tempo_emprego + qt_pessoas_residencia + 1', data = renda_prev)
x

DesignMatrix with shape (12427, 40)
  Columns:
    ['Intercept',
     'C(data_ref)[T.2015-02-01]',
     'C(data_ref)[T.2015-03-01]',
     'C(data_ref)[T.2015-04-01]',
     'C(data_ref)[T.2015-05-01]',
     'C(data_ref)[T.2015-06-01]',
     'C(data_ref)[T.2015-07-01]',
     'C(data_ref)[T.2015-08-01]',
     'C(data_ref)[T.2015-09-01]',
     'C(data_ref)[T.2015-10-01]',
     'C(data_ref)[T.2015-11-01]',
     'C(data_ref)[T.2015-12-01]',
     'C(data_ref)[T.2016-01-01]',
     'C(data_ref)[T.2016-02-01]',
     'C(data_ref)[T.2016-03-01]',
     'C(sexo)[T.M]',
     'posse_de_veiculo[T.True]',
     'posse_de_imovel[T.True]',
     'C(tipo_renda)[T.Bolsista]',
     'C(tipo_renda)[T.Empresário]',
     'C(tipo_renda)[T.Pensionista]',
     'C(tipo_renda)[T.Servidor público]',
     'C(educacao)[T.Pós graduação]',
     'C(educacao)[T.Secundário]',
     'C(educacao)[T.Superior completo]',
     'C(educacao)[T.Superior incompleto]',
     'C(estado_civil, Treatment())[T.Separado]',
     'C(estado_civil

In [43]:

frequencia = renda_prev.value_counts()
categoria_mais_frequente = frequencia.idxmax()

print(f"A categoria mais frequente é: {categoria_mais_frequente}")

A categoria mais frequente é: (0, '2015-01-01', 15056, 'F', False, True, 0, 'Empresário', 'Secundário', 'Solteiro', 'Casa', 26, 6.602739726027397, 1.0, 8060.34)


In [44]:
sm.OLS(y, x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.257
Model:                            OLS   Adj. R-squared:                  0.255
Method:                 Least Squares   F-statistic:                     109.8
Date:                Mon, 03 Feb 2025   Prob (F-statistic):               0.00
Time:                        17:56:33   Log-Likelihood:            -1.2878e+05
No. Observations:               12427   AIC:                         2.576e+05
Df Residuals:                   12387   BIC:                         2.579e+05
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
============================================================================================================
                                               coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
Intercept                                -5759.2442   2520.039     -2.285      0.022   -1.07e+04    -819.575
C(data_ref)[T.2015-02-01]                  -51.9151    377.096     -0.138      0.891    -791.082     687.252
C(data_ref)[T.2015-03-01]                  222.9354    378.084      0.590      0.555    -518.168     964.039
C(data_ref)[T.2015-04-01]                  642.3659    376.953      1.704      0.088     -96.520    1381.252
C(data_ref)[T.2015-05-01]                  176.5809    375.914      0.470      0.639    -560.270     913.432
C(data_ref)[T.2015-06-01]                  642.6161    377.479      1.702      0.089     -97.302    1382.534
C(data_ref)[T.2015-07-01]                  366.5612    376.191      0.974      0.330    -370.831    1103.954
C(data_ref)[T.2015-08-01]                  207.0676    377.492      0.549      0.583    -532.875     947.010
C(data_ref)[T.2015-09-01]                  150.6806    374.958      0.402      0.688    -584.296     885.657
C(data_ref)[T.2015-10-01]                  272.5373    377.511      0.722      0.470    -467.442    1012.517
C(data_ref)[T.2015-11-01]                  444.1329    377.686      1.176      0.240    -296.190    1184.455
C(data_ref)[T.2015-12-01]                  624.4096    378.999      1.648      0.099    -118.488    1367.307
C(data_ref)[T.2016-01-01]                  302.4136    377.447      0.801      0.423    -437.441    1042.268
C(data_ref)[T.2016-02-01]                  -43.5417    377.367     -0.115      0.908    -783.239     696.155
C(data_ref)[T.2016-03-01]                   60.1761    380.722      0.158      0.874    -686.098     806.450
C(sexo)[T.M]                              6056.3382    155.940     38.838      0.000    5750.672    6362.005
posse_de_veiculo[T.True]                    42.9217    150.594      0.285      0.776    -252.267     338.110
posse_de_imovel[T.True]                    352.5163    149.455      2.359      0.018      59.561     645.472
C(tipo_renda)[T.Bolsista]                -1377.8362   2568.569     -0.536      0.592   -6412.631    3656.958
C(tipo_renda)[T.Empresário]                828.6254    158.916      5.214      0.000     517.125    1140.126
C(tipo_renda)[T.Pensionista]             -2632.1209   2569.530     -1.024      0.306   -7668.798    2404.557
C(tipo_renda)[T.Servidor público]          138.3702    236.531      0.585      0.559    -325.267     602.007
C(educacao)[T.Pós graduação]              1010.8977   1691.642      0.598      0.550   -2304.984    4326.780
C(educacao)[T.Secundário]                  560.1259    766.269      0.731      0.465    -941.881    2062.133
C(educacao)[T.Superior completo]          1177.9727    769.202      1.531      0.126    -329.783    2685.729
C(educacao)[T.Superior incomple

#### 1.3 

Vários dos parâmetros não possuem relevância. O primeiro que pensei a ser excluído do modelo é a data, acredito que não faria sentido nessa caso ajustar um modelo com base em um mês específico de um dataframe, pensando em utilizá-lo em outro contexto, ou outro conjunto de dados. Fiz um primeiro teste e o modelo piorou, então tirei outra variável cujas categorias não tinham significância, o tipo de residência.

De todos os meses do DF apenas 2 têm significância, então vou excluir a data pra parte 2. 

In [45]:
y, x = patsy.dmatrices('renda ~ C(data_ref) + id_cliente + C(sexo) + posse_de_veiculo + posse_de_imovel +'
                       ' qtd_filhos + C(tipo_renda) + C(educacao) + C(estado_civil, Treatment()) +'
                       ' idade + tempo_emprego + qt_pessoas_residencia + 1', data = renda_prev)

sm.OLS(y, x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.257
Model:                            OLS   Adj. R-squared:                  0.255
Method:                 Least Squares   F-statistic:                     126.0
Date:                Mon, 03 Feb 2025   Prob (F-statistic):               0.00
Time:                        17:56:40   Log-Likelihood:            -1.2878e+05
No. Observations:               12427   AIC:                         2.576e+05
Df Residuals:                   12392   BIC:                         2.579e+05
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
============================================================================================================
                                               coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
Intercept                                -6064.6540   2458.214     -2.467      0.014   -1.09e+04   -1246.172
C(data_ref)[T.2015-02-01]                  -44.0508    376.906     -0.117      0.907    -782.845     694.743
C(data_ref)[T.2015-03-01]                  231.5993    377.940      0.613      0.540    -509.221     972.420
C(data_ref)[T.2015-04-01]                  643.4377    376.880      1.707      0.088     -95.306    1382.181
C(data_ref)[T.2015-05-01]                  179.0504    375.814      0.476      0.634    -557.604     915.705
C(data_ref)[T.2015-06-01]                  650.1269    377.345      1.723      0.085     -89.529    1389.782
C(data_ref)[T.2015-07-01]                  376.3992    375.981      1.001      0.317    -360.583    1113.381
C(data_ref)[T.2015-08-01]                  216.7601    377.315      0.574      0.566    -522.837     956.357
C(data_ref)[T.2015-09-01]                  151.5672    374.846      0.404      0.686    -583.188     886.323
C(data_ref)[T.2015-10-01]                  281.7391    377.309      0.747      0.455    -457.845    1021.323
C(data_ref)[T.2015-11-01]                  446.3835    377.567      1.182      0.237    -293.706    1186.473
C(data_ref)[T.2015-12-01]                  634.7800    378.810      1.676      0.094    -107.746    1377.306
C(data_ref)[T.2016-01-01]                  315.0635    377.189      0.835      0.404    -424.286    1054.413
C(data_ref)[T.2016-02-01]                  -37.8320    377.244     -0.100      0.920    -777.288     701.624
C(data_ref)[T.2016-03-01]                   65.5855    380.566      0.172      0.863    -680.382     811.553
C(sexo)[T.M]                              6059.6880    155.497     38.970      0.000    5754.890    6364.486
posse_de_veiculo[T.True]                    43.2417    150.500      0.287      0.774    -251.761     338.245
posse_de_imovel[T.True]                    339.5691    146.636      2.316      0.021      52.139     626.999
C(tipo_renda)[T.Bolsista]                -1388.4685   2568.143     -0.541      0.589   -6422.428    3645.491
C(tipo_renda)[T.Empresário]                833.7464    158.639      5.256      0.000     522.790    1144.703
C(tipo_renda)[T.Pensionista]             -2639.9601   2568.800     -1.028      0.304   -7675.208    2395.288
C(tipo_renda)[T.Servidor público]          140.7835    236.281      0.596      0.551    -322.364     603.931
C(educacao)[T.Pós graduação]               997.7024   1690.044      0.590      0.555   -2315.046    4310.451
C(educacao)[T.Secundário]                  559.1110    763.882      0.732      0.464    -938.217    2056.439
C(educacao)[T.Superior completo]          1171.3762    767.307      1.527      0.127    -332.665    2675.417
C(educacao)[T.Superior incomple

O $Rˆ2$ não mudou nem o $Rˆ2$ ajustado. O AIC também não mudou, ou seja, não houve alteração perceptível na qualidade do modelo.

In [52]:
y1, x1 = patsy.dmatrices('renda ~ + C(sexo) + posse_de_imovel +'
                       ' idade + tempo_emprego + 1', data = renda_prev)

sm.OLS(y1, x1).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.252
Model:                            OLS   Adj. R-squared:                  0.252
Method:                 Least Squares   F-statistic:                     1047.
Date:                Mon, 03 Feb 2025   Prob (F-statistic):               0.00
Time:                        19:35:45   Log-Likelihood:            -1.2882e+05
No. Observations:               12427   AIC:                         2.577e+05
Df Residuals:                   12422   BIC:                         2.577e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept               -2121.6370    329.140     -6.446      0.000   -2766.803   -1476.471
C(sexo)[T.M]             6045.7894    145.591     41.526      0.000    5760.409    6331.169
posse_de_imovel[T.True]   394.7377    145.657      2.710      0.007     109.228     680.247
idade                      37.5533      7.838      4.791      0.000      22.189      52.918
tempo_emprego             550.4590     10.903     50.485      0.000     529.087     571.831
==============================================================================
Omnibus:                    17506.453   Durbin-Watson:                   2.036
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          9670624.952
Skew:                           8.073   Prob(JB):                         0.00
Kurtosis:                     138.705   Cond. No.                         206.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

O modelo piorou com relação ao Rˆ2, O AIC também aumentou. Vou tentar alguns ajustes de funções.

In [55]:
y2, x2 = patsy.dmatrices('np.log(renda) ~ + C(sexo) + posse_de_imovel +'
                       ' idade + np.log(tempo_emprego) + 1', data = renda_prev)

sm.OLS(y2, x2).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.310
Model:                            OLS   Adj. R-squared:                  0.309
Method:                 Least Squares   F-statistic:                     1392.
Date:                Mon, 03 Feb 2025   Prob (F-statistic):               0.00
Time:                        19:38:01   Log-Likelihood:                -14015.
No. Observations:               12427   AIC:                         2.804e+04
Df Residuals:                   12422   BIC:                         2.808e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   6.9747      0.032    215.264      0.000       6.911       7.038
C(sexo)[T.M]                0.7669      0.014     54.270      0.000       0.739       0.795
posse_de_imovel[T.True]     0.0788      0.014      5.563      0.000       0.051       0.107
idade                       0.0093      0.001     12.491      0.000       0.008       0.011
np.log(tempo_emprego)       0.3565      0.007     51.678      0.000       0.343       0.370
==============================================================================
Omnibus:                       11.504   Durbin-Watson:                   2.025
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               11.487
Skew:                           0.073   Prob(JB):                      0.00320
Kurtosis:                       3.030   Cond. No.                         205.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Comentário

O melhor ajuste foi uma relação ao log da variável target e da principal variável explicativa (tempo_emprego).